# Prompt Templates & Output parsers

In [4]:
import sys
sys.path.insert(1, '../../')
import init_creds as creds
 
AZURE_OPENAI_API_KEY = creds.get_api_key()
AZURE_OPENAI_ENDPOINT = creds.get_endpoint()
print(AZURE_OPENAI_API_KEY)
print(AZURE_OPENAI_ENDPOINT)
 
if not AZURE_OPENAI_API_KEY:
    raise ValueError("No AZURE_OPENAI_API_KEY set for Azure OpenAI API")
if not AZURE_OPENAI_ENDPOINT:
    raise ValueError("No AZURE_OPENAI_ENDPOINT set for Azure OpenAI API")

In [5]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o-mini"
os.environ["AZURE_OPENAI_API_VERSION"]="2024-07-01-preview"

print(f'AZURE_OPENAI_API_KEY={os.environ["AZURE_OPENAI_API_KEY"]}')
print(f'AZURE_OPENAI_ENDPOINT={os.environ["AZURE_OPENAI_ENDPOINT"]}')
print(f'AZURE_OPENAI_DEPLOYMENT_NAME={os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]}')
print(f'AZURE_OPENAI_API_VERSION={os.environ["AZURE_OPENAI_API_VERSION"]}'    )


from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [8]:
from langchain_core.prompts import ChatPromptTemplate
system_template = """
User loves going for trips and have gone to many different places in {place} and plans to visit few more soon.
Can you create a tweet with {word_count} words for above?
"""
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{place}")]
)
formatted_prompt = prompt_template.invoke({"word_count": "25", "place": "diego garcia"})

model.invoke(formatted_prompt).content

In [1]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
examples=[
    {
        "query":"What is a headphone?",
        "answer":"Its a device that helps to hear and have fun."
    },
    {
        "query":"What is a planet?",
        "answer":"It's a sphere that goes around sun and gets tired."
    }
]



# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{query}"),
        ("ai", "{answer}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a cheeky four-year-old who is smart and funny"),
        few_shot_prompt,
        ("human", "{user_query}"),
    ]
)
user_query="What is a watch?"
model.invoke(final_prompt.format(user_query=user_query)).content

NameError: name 'model' is not defined

In [12]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.messages import HumanMessage, SystemMessage
output_parser=CommaSeparatedListOutputParser()
format_instructions=output_parser.get_format_instructions()
format_instructions


messages = [
    SystemMessage(content="Help answer user questions in"+format_instructions),
    HumanMessage(content="Give me examples of five birds"),
]
model.invoke(messages).content
output_parser.parse(model.invoke(messages).content)

In [13]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.messages import HumanMessage, SystemMessage
response_schemas=[
    ResponseSchema(name="Country",description="Answer to user's query"),
    ResponseSchema(name="ShortName",description="Short name of the country")
]
output_parser=StructuredOutputParser.from_response_schemas(response_schemas)
formatjson_instructions=output_parser.get_format_instructions()
messages = [
    SystemMessage(content="Help user answer user questions in"+formatjson_instructions),
    HumanMessage(content="Oldest country in the world"),
]

output_parser.parse(model.invoke(messages).content)

## Python Type Annotations

The `->` symbol in Python is used for function return type annotations, introduced in Python 3.5 (PEP 484).

### Basic Syntax:
```python
def function_name(parameter: type) -> return_type:
    # function body
```

### Examples:
```python
# Function that takes a string and returns a string
def greeting(name: str) -> str:
    return f"Hello, {name}"

# Function that takes two integers and returns an integer
def add(a: int, b: int) -> int:
    return a + b
    
# Function that returns nothing (None)
def log_message(message: str) -> None:
    print(f"Log: {message}")
```

Type annotations are optional in Python and don't affect runtime behavior. They're useful for:
- Documentation
- IDE autocompletion and error checking
- Static type checking with tools like mypy
- Better code readability

In LangChain code, you'll often see type annotations used to specify return types from various functions and methods.